#### **This notebook is used for HiC data processing**

In [ ]:
import pandas as pd
import os
from pathos.multiprocessing import ProcessingPool as Pool

**Process each cell line hic data(fq: mean, fdr: mean, rawc: "first")**

In [ ]:
file_path = "../Data/all_processed_HiC/K_all_hic_cont.csv.gz"
df = pd.read_csv(file_path)
df.head()

,chr,ibp,jbp,fq,fdr,rawc
0,chr1,69740000,69780000,0.780465,0.0,29.353172
1,chr1,69740000,69790000,0.370194,0.0,22.741314
2,chr1,69740000,69950000,0.068101,0.0,12.943521
3,chr1,69740000,71210000,0.035838,0.0,10.222382
4,chr1,69745000,69800000,0.531329,0.0,25.584558


In [ ]:
df_refined_GM = pd.read_csv("../Data/all_processed_HiC/GM_all_hic_cont.csv.gz")
df_refined_GM = df_refined_GM.groupby(["chr", "ibp", "jbp"], as_index=False).agg(
    {"fq": "mean", "fdr": "mean", "rawc": "first"}
)

In [5]:
df_refined_GM.head()

,chr,ibp,jbp,fq,fdr,rawc
0,chr1,3320000,3320000,1.000000,1.000000,156.021600
1,chr1,3320000,3325000,1.000000,1.000000,59.187775
2,chr1,3320000,3330000,1.000000,1.000000,39.219370
3,chr1,3320000,3335000,0.998299,0.981670,24.210155
4,chr1,3320000,3340000,0.809792,0.855084,20.779170


In [ ]:
""" Batch process each cell line HiC data """


data_dir = "../Data/all_processed_HiC"
out_put_dir = "../Data/refined_processed_HiC"

files = [f for f in os.listdir(data_dir) if f.endswith(".csv.gz")]

for file in files:
    # cell line name
    cell_line = file.split("_")[0]

    file_path = os.path.join(data_dir, file)
    df = pd.read_csv(file_path)

    # get the average value of 'fq' and 'fdr' for each group
    df = df.groupby(["chr", "ibp", "jbp"], as_index=False).agg(
        {"fq": "mean", "fdr": "mean", "rawc": "first"}
    )

    df["cell_line"] = cell_line

    # save the processed HiC file
    processed_file_path = os.path.join(out_put_dir, f"{cell_line}_processed.csv.gz")
    df.to_csv(processed_file_path, index=False, compression="gzip")
    print(f"Processed file saved to {processed_file_path}")

Processed file saved to ../Data/refined_processed_HiC/K_processed.csv.gz
Processed file saved to ../Data/refined_processed_HiC/IMR_processed.csv.gz
Processed file saved to ../Data/refined_processed_HiC/GM_processed.csv.gz


In [5]:
file_path = "../Data/refined_processed_HiC/IMR_processed.csv.gz"
IMR_df = pd.read_csv(file_path)
print(IMR_df.tail())

            chr        ibp        jbp   fq  fdr        rawc cell_line
121150427  chrX  154055000  154060000  1.0  1.0   98.492010       IMR
121150428  chrX  154055000  154065000  1.0  1.0   59.830734       IMR
121150429  chrX  154060000  154060000  1.0  1.0  187.421430       IMR
121150430  chrX  154060000  154065000  1.0  1.0  102.419624       IMR
121150431  chrX  154065000  154065000  1.0  1.0  198.546620       IMR


#### **THIS IS A OLD RNAGES, YOU MIGHT BE NOT WANT TO USE IT**

**Find each cell line chromosome's range**

In [ ]:
def process_file(args):
    input_folder, filename, output_folder = args
    if not filename.endswith(".csv.gz"):
        return

    cell_line = filename.split("_")[0]
    file_path = os.path.join(input_folder, filename)
    df = pd.read_csv(file_path, compression="gzip")

    df["ibp_end"] = df["ibp"] + 5000
    df["jbp_end"] = df["jbp"] + 5000

    # batch processing
    ibp_ranges = df[["chr", "ibp", "ibp_end"]].rename(columns={"ibp": "start_value", "ibp_end": "end_value"})
    jbp_ranges = df[["chr", "jbp", "jbp_end"]].rename(columns={"jbp": "start_value", "jbp_end": "end_value"})
    ranges = pd.concat([ibp_ranges, jbp_ranges])
    ranges["cell_line"] = cell_line

    ranges = ranges.drop_duplicates().sort_values(by=["chr", "start_value", "end_value"])

    # merge adjacent ranges
    merged_results = []
    for chr_id, group in ranges.groupby("chr"):
        group = group.sort_values(by="start_value").to_dict(orient="records")
        merged = [group[0]]

        for entry in group[1:]:
            last = merged[-1]
            if last["end_value"] >= entry["start_value"]:
                last["end_value"] = max(last["end_value"], entry["end_value"])
            else:
                merged.append(entry)

        for entry in merged:
            merged_results.append({
                "cell_line": cell_line,
                "chrID": chr_id,
                "start_value": entry["start_value"],
                "end_value": entry["end_value"],
            })

    result_df = pd.DataFrame(merged_results, columns=["cell_line", "chrID", "start_value", "end_value"])

    # save
    output_filename = f"{cell_line}_ranges.csv.gz"
    output_path = os.path.join(output_folder, output_filename)
    result_df.to_csv(output_path, index=False, compression="gzip")
    print(f"Processed and saved: {output_filename}")

def main(input_folder, output_folder, num_workers=4):
    files = [(input_folder, filename, output_folder) for filename in os.listdir(input_folder) if filename.endswith(".csv.gz")]

    # process files in parallel
    with Pool(num_workers) as pool:
        pool.map(process_file, files)

input_folder = "./refined_processed_HiC"
output_folder = "./seqs"


main(input_folder, output_folder)

In [6]:
file_path = "../Example_Data/seqs/K_ranges.csv.gz"
IMR_df = pd.read_csv(file_path)

print(f"Total rows: {len(IMR_df)}")
print(IMR_df)

equal_values_df = IMR_df[IMR_df["start_value"] == IMR_df["end_value"]]

print(f"Rows where start_value == end_value: {len(equal_values_df)}")

if len(equal_values_df) > 0:
    print("Rows where start_value == end_value:")
    print(equal_values_df)
else:
    print("No rows where start_value == end_value found.")


Total rows: 44
   cell_line  chrID  start_value  end_value
0          K  chr12       315000    8410000
1          K  chr12      8415000    8425000
2          K  chr12      8435000    9515000
3          K  chr12      9530000    9555000
4          K  chr12      9565000   11360000
5          K  chr12     11370000   11375000
6          K  chr12     11390000   34510000
7          K  chr12     39180000  132600000
8          K  chr17       605000   12450000
9          K  chr17     12455000   18460000
10         K  chr17     18465000   19155000
11         K  chr17     19160000   19165000
12         K  chr17     19180000   19190000
13         K  chr17     19205000   19215000
14         K  chr17     19220000   21800000
15         K  chr17     21810000   21905000
16         K  chr17     21920000   21925000
17         K  chr17     21930000   21955000
18         K  chr17     21965000   21995000
19         K  chr17     22040000   22520000
20         K  chr17     27085000   36380000
21         K  chr